### Test trivial case, to see if I can access Tiingo's Restful API

### Tiingo Documentation:
- Data rate limits for free tier: https://www.tiingo.com/about/pricing
- How to get end-of-day prices: https://www.tiingo.com/documentation/end-of-day

In [1]:
# Standard library imports
import sys
import os

from dotenv import load_dotenv
import requests

# /home/ubuntu/financial-etl-poc/this_folder
current_folder = os.getcwd()

# /home/ubuntu/financial-etl-poc/
project_root_folder = os.path.abspath(os.path.join(current_folder, ".."))
sys.path.append(project_root_folder)

# For loading credentials from .env under financial-etl-poc
dotenv_path = os.path.join(project_root_folder, ".env")
load_dotenv(dotenv_path)

True

In [ ]:
tiingo_api_token = os.getenv("tiingo_api_token")
tiingo_url = f"https://api.tiingo.com/api/test?token={tiingo_api_token}"

headers = {'Content-Type': 'application/json'}
requestResponse = requests.get(tiingo_url, headers = headers)
requestResponse.json()

{'message': 'You successfully sent a request'}

### Test non-trivial case for one stock and a few dates

In [ ]:
# Test to see how Tiingo handles dividends and stock splits
# Nvidia was $120.89 on 2024-06-07 after adjustment. This was before a 10:1 stock split and a few dividend payments

start_date = "2024-06-07"
end_date = "2024-06-07"
ticker = "NVDA"
freq = "daily"
format = "json"

url = f"https://api.tiingo.com/tiingo/daily/{ticker}/prices?startDate={start_date}&endDate={end_date}&resampleFreq={freq}&format={format}&token={tiingo_api_token}"

params = {
    "startDate": start_date,
    "endDate": end_date,
    "format": "csv",
    "resampleFreq": "daily"
}

headers = {'Content-Type': 'application/json'}

response = requests.get(url, headers=headers)
response.json()

[{'date': '2024-06-07T00:00:00.000Z',
  'close': 1208.88,
  'high': 1216.9171,
  'low': 1180.22,
  'open': 1197.7,
  'volume': 41238580,
  'adjClose': 120.849084499,
  'adjHigh': 121.6525357738,
  'adjLow': 117.9840071036,
  'adjOpen': 119.7314443985,
  'adjVolume': 412385800,
  'divCash': 0.0,
  'splitFactor': 1.0}]

In [13]:
# Cleaner version
start_date = "2024-06-07"
end_date = "2024-06-12"
ticker = "NVDA"

url = f"https://api.tiingo.com/tiingo/daily/{ticker}/prices"

params = {
    "startDate": start_date,
    "endDate": end_date,
    "resampleFreq": "daily",
    "format": "json",
    "token": tiingo_api_token
}

headers = {'Content-Type': 'application/json'}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()  # always good for catching HTTP errors
response.json()

[{'date': '2024-06-07T00:00:00.000Z',
  'close': 1208.88,
  'high': 1216.9171,
  'low': 1180.22,
  'open': 1197.7,
  'volume': 41238580,
  'adjClose': 120.849084499,
  'adjHigh': 121.6525357738,
  'adjLow': 117.9840071036,
  'adjOpen': 119.7314443985,
  'adjVolume': 412385800,
  'divCash': 0.0,
  'splitFactor': 1.0},
 {'date': '2024-06-10T00:00:00.000Z',
  'close': 121.79,
  'high': 123.1,
  'low': 117.01,
  'open': 120.37,
  'volume': 308134791,
  'adjClose': 121.7507941329,
  'adjHigh': 123.060372426,
  'adjLow': 116.9723328803,
  'adjOpen': 120.3312512503,
  'adjVolume': 308134791,
  'divCash': 0.0,
  'splitFactor': 10.0},
 {'date': '2024-06-11T00:00:00.000Z',
  'close': 120.91,
  'high': 122.87,
  'low': 118.74,
  'open': 121.77,
  'volume': 222551158,
  'adjClose': 120.8810741978,
  'adjHigh': 122.8406052988,
  'adjLow': 118.7115933359,
  'adjOpen': 121.7408684564,
  'adjVolume': 222551158,
  'divCash': 0.01,
  'splitFactor': 1.0},
 {'date': '2024-06-12T00:00:00.000Z',
  'close': 